# Objective

To review model selection and model assessment in a classification context


## Preliminaries

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.ensemble import VotingClassifier

**Model selection** involves estimating $E_{val}$ for a bouqet of models in order to arrive at a final model

**Model assessment** involves estimating $E_{test}$ for the *final model* before it is rolled into production

Model selection requires a validation set

Model assessment required a test set

# Example: **Airplane accidents**

## Data

In [2]:
airplanes_df = pd.read_csv("/content/drive/MyDrive/AI-ML/supervised-learning-revision/Day2/data/airplane-accidents.csv")

In [3]:
airplanes_df.shape

(10000, 12)

In [4]:
airplanes_df.head()

,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
0,Minor_Damage_And_Injuries,49.223744,14,22,71.285324,0.272118,78.04,2,31335.476824,3,0.424352,7570
1,Minor_Damage_And_Injuries,62.465753,10,27,72.288058,0.423939,84.54,2,26024.711057,2,0.352350,12128
2,Significant_Damage_And_Fatalities,63.059361,13,16,66.362808,0.322604,78.86,7,39269.053927,3,0.003364,2181
3,Significant_Damage_And_Serious_Injuries,48.082192,11,9,74.703737,0.337029,81.79,3,42771.499200,1,0.211728,5946
4,Significant_Damage_And_Fatalities,26.484018,13,25,47.948952,0.541140,77.16,3,35509.228515,2,0.176883,9054


In [5]:
airplanes_X, airplanes_y = (airplanes_df.drop(["Severity", "Accident_ID"], axis=1),
                            airplanes_df.Severity)

In [6]:
airplanes_X_train, airplanes_X_test, airplanes_y_train, airplanes_y_test = train_test_split(airplanes_X,
                                                                                            airplanes_y,
                                                                                            test_size=0.2,
                                                                                            random_state=20130810)

We will not touch testing data till the very end.

## Model 1: **Softmax**

In [7]:
airplanes_Xtrain, airplanes_Xvalid, airplanes_ytrain, airplanes_yvalid = train_test_split(airplanes_X_train,
                                                                                          airplanes_y_train,
                                                                                          test_size=0.2,
                                                                                          random_state=20130810)

### Tuning *C*


In [8]:
sc = StandardScaler()

In [9]:
airplanes_scaledXtrain = sc.fit_transform(airplanes_Xtrain)

airplanes_scaledXvalid = sc.transform(airplanes_Xvalid)

In [10]:
C_values = [0.001, 0.1, 1, 5, 10]

In [11]:
training_accuracy, validation_accuracy = [], []
training_f1, validation_f1 = [], []

In [12]:
%%time

for c in C_values:
  learner_softmax = LogisticRegression(C=c,
                                       multi_class='multinomial')
  
  learner_softmax.fit(airplanes_scaledXtrain, 
                      airplanes_ytrain)
  
  training_accuracy.append(learner_softmax.score(airplanes_scaledXtrain,
                                                 airplanes_ytrain))
  
  validation_accuracy.append(learner_softmax.score(airplanes_scaledXvalid,
                                                   airplanes_yvalid))
  
  training_f1.append(f1_score(airplanes_ytrain,
                              learner_softmax.predict(airplanes_scaledXtrain),
                              average='macro'))
  
  validation_f1.append(f1_score(airplanes_yvalid,
                                learner_softmax.predict(airplanes_scaledXvalid),
                                average='macro'))

CPU times: user 496 ms, sys: 0 ns, total: 496 ms
Wall time: 512 ms


In [13]:
pd.DataFrame({'C': C_values,
              'training_accuracy': training_accuracy,
              'validation_accuracy': validation_accuracy,
              'training_f1': training_f1,
              'validation_f1': validation_f1})

,C,training_accuracy,validation_accuracy,training_f1,validation_f1
0,0.001,0.505469,0.49625,0.437130,0.423211
1,0.100,0.640469,0.62750,0.595043,0.574621
2,1.000,0.644687,0.63125,0.599743,0.579201
3,5.000,0.644844,0.63125,0.599886,0.579116
4,10.000,0.644844,0.63125,0.599886,0.579116


The best model has $C=1$

## Model 2: **KNN**

In [14]:
airplanes_Xtrain, airplanes_Xvalid, airplanes_ytrain, airplanes_yvalid = train_test_split(airplanes_X_train,
                                                                                          airplanes_y_train,
                                                                                          test_size=0.2,
                                                                                          random_state=20130810)

In [15]:
sc = StandardScaler()

In [16]:
airplanes_scaledXtrain = sc.fit_transform(airplanes_Xtrain)

airplanes_scaledXvalid = sc.transform(airplanes_Xvalid)

### Tuning *$K$*

In [17]:
k_values = [2, 3, 4, 5, 6, 7, 8, 9, 10]

training_accuracy, validation_accuracy = [], []
training_f1, validation_f1 = [], []

In [18]:
%%time

for k in k_values:
  learner_knn = KNeighborsClassifier(n_neighbors=k)
  learner_knn.fit(airplanes_scaledXtrain, airplanes_ytrain)

  training_accuracy.append(learner_knn.score(airplanes_scaledXtrain,
                                             airplanes_ytrain))

  validation_accuracy.append(learner_knn.score(airplanes_scaledXvalid, 
                                               airplanes_yvalid))
  
  training_f1.append(f1_score(airplanes_yvalid,
                              learner_knn.predict(airplanes_scaledXvalid),
                              average='macro'))
  
  validation_f1.append(f1_score(airplanes_yvalid,
                                learner_knn.predict(airplanes_scaledXvalid),
                                average='macro')) 

CPU times: user 7.94 s, sys: 0 ns, total: 7.94 s
Wall time: 7.98 s


In [19]:
pd.DataFrame({'k': k_values,
              'training_accuracy': training_accuracy,
              'validation_accuracy': validation_accuracy,
              'training_f1': training_f1,
              'validation_f1': validation_f1})

,k,training_accuracy,validation_accuracy,training_f1,validation_f1
0,2,0.848437,0.648750,0.633018,0.633018
1,3,0.847500,0.678750,0.676753,0.676753
2,4,0.825000,0.700000,0.695395,0.695395
3,5,0.803750,0.686875,0.683171,0.683171
4,6,0.798281,0.705000,0.698091,0.698091
5,7,0.786563,0.692500,0.687447,0.687447
6,8,0.781563,0.706250,0.699183,0.699183
7,9,0.772344,0.696250,0.692693,0.692693
8,10,0.770469,0.691875,0.684567,0.684567


The best model has $k=8$.

## Model 3: **Naive Bayes**

In [20]:
airplanes_Xtrain, airplanes_Xvalid, airplanes_ytrain, airplanes_yvalid = train_test_split(airplanes_X_train,
                                                                                          airplanes_y_train,
                                                                                          test_size=0.2,
                                                                                          random_state=20130810)

In [21]:
sc = StandardScaler()

In [22]:
airplanes_scaledXtrain = sc.fit_transform(airplanes_Xtrain)

airplanes_scaledXvalid = sc.transform(airplanes_Xvalid)

In [23]:
learner_multinomialnb = GaussianNB()

In [24]:
learner_multinomialnb.fit(airplanes_scaledXtrain,
                          airplanes_ytrain)

GaussianNB(priors=None, var_smoothing=1e-09)

Validation accuracy

In [25]:
learner_multinomialnb.score(airplanes_scaledXvalid,
                            airplanes_yvalid)

0.458125

Validation F1

In [26]:
f1_score(airplanes_yvalid,
         learner_multinomialnb.predict(airplanes_scaledXvalid),
         average='macro')

0.43115730298558425

## Comparing between best learners within each algorithm

In [27]:
sc = StandardScaler()

Go back to the original train data

In [28]:
airplanes_scaledXtrain = sc.fit_transform(airplanes_X_train)

Usually a cross validation approach is computationally more approachable at this stage.

In [29]:
learner_best_softmax = LogisticRegression(C=1, multi_class='multinomial')
learner_best_knn = KNeighborsClassifier(n_neighbors=8)
learner_best_gaussiannb = GaussianNB()

In [30]:
cross_val_score(learner_best_softmax,
                airplanes_scaledXtrain,
                y=airplanes_y_train,
                cv=10,
                scoring='f1_macro')

array([0.58706611, 0.57749395, 0.59441124, 0.60192975, 0.57857361,
       0.63338168, 0.57958805, 0.62135056, 0.58784564, 0.56713612])

In [31]:
cross_val_score(learner_best_knn,
                airplanes_scaledXtrain,
                y=airplanes_y_train,
                cv=10,
                scoring='f1_macro')

array([0.69426131, 0.67489772, 0.69893028, 0.68736816, 0.71431292,
       0.72674767, 0.68706051, 0.70839449, 0.69800033, 0.71457742])

In [32]:
cross_val_score(learner_best_gaussiannb,
                airplanes_scaledXtrain,
                y=airplanes_y_train,
                cv=10,
                scoring='f1_macro')

array([0.4239052 , 0.43661353, 0.41520808, 0.45365884, 0.42886841,
       0.47908403, 0.43798537, 0.47997675, 0.41757438, 0.42611304])

## Final model

In [33]:
sc = StandardScaler()

airplanes_scaledXtrain = sc.fit_transform(airplanes_X_train)
airplanes_scaledXtest = sc.transform(airplanes_X_test)

In [34]:
learner_best_knn.fit(airplanes_scaledXtrain,
                     airplanes_y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=8, p=2,
                     weights='uniform')

In [35]:
f1_score(airplanes_y_test,
         learner_best_knn.predict(airplanes_scaledXtest),
         average='macro')

0.6916705981893833

# Bonus



## A simple ensemble

In [36]:
airplanes_Xtrain, airplanes_Xvalid, airplanes_ytrain, airplanes_yvalid = train_test_split(airplanes_X_train,
                                                                                          airplanes_y_train,
                                                                                          test_size=0.2,
                                                                                          random_state=20130810)

In [37]:
sc = StandardScaler()

In [38]:
airplanes_scaledXtrain = sc.fit_transform(airplanes_Xtrain)

airplanes_scaledXvalid = sc.transform(airplanes_Xvalid)

In [39]:
learner_best_softmax.fit(airplanes_scaledXtrain, airplanes_ytrain)
learner_best_knn.fit(airplanes_scaledXtrain, airplanes_ytrain)
learner_best_gaussiannb.fit(airplanes_scaledXtrain, airplanes_ytrain)

GaussianNB(priors=None, var_smoothing=1e-09)

In [40]:
learner_best_softmax.predict_proba(airplanes_scaledXvalid)

array([[0.04882772, 0.19623567, 0.21608242, 0.53885419],
       [0.01180619, 0.27347744, 0.41700822, 0.29770815],
       [0.65670733, 0.0465665 , 0.02264697, 0.2740792 ],
       ...,
       [0.53967525, 0.03602797, 0.10212034, 0.32217643],
       [0.00614702, 0.23615936, 0.52170778, 0.23598585],
       [0.61353996, 0.06017378, 0.01582984, 0.31045642]])

In [41]:
learner_best_knn.predict_proba(airplanes_scaledXvalid)

array([[0.125, 0.375, 0.   , 0.5  ],
       [0.   , 0.25 , 0.75 , 0.   ],
       [0.75 , 0.   , 0.   , 0.25 ],
       ...,
       [0.375, 0.   , 0.   , 0.625],
       [0.   , 0.25 , 0.5  , 0.25 ],
       [0.5  , 0.125, 0.   , 0.375]])

In [42]:
learner_best_gaussiannb.predict_proba(airplanes_scaledXvalid)

array([[0.04033574, 0.1810391 , 0.30460975, 0.4740154 ],
       [0.04267425, 0.11489358, 0.29313437, 0.5492978 ],
       [0.32620806, 0.14032347, 0.0351204 , 0.49834807],
       ...,
       [0.26833358, 0.06327004, 0.16753335, 0.50086303],
       [0.03109257, 0.1367536 , 0.45829285, 0.37386098],
       [0.15650024, 0.36891169, 0.04428082, 0.43030725]])

In [43]:
(0.3 * learner_best_softmax.predict_proba(airplanes_scaledXvalid) +
 0.5 * learner_best_knn.predict_proba(airplanes_scaledXvalid) +
 0.2 * learner_best_gaussiannb.predict_proba(airplanes_scaledXvalid))

array([[0.08521547, 0.28257852, 0.12574668, 0.50645934],
       [0.01207671, 0.23002195, 0.55872934, 0.199172  ],
       [0.63725381, 0.04203464, 0.01381817, 0.30689337],
       ...,
       [0.40306929, 0.0234624 , 0.06414277, 0.50932554],
       [0.00806262, 0.22319853, 0.4981709 , 0.27056795],
       [0.46536204, 0.15433447, 0.01360512, 0.36669837]])

This idea of working with weighted probability score from multiple estimators is implemented in `scikit-learn` in the `VotingClassifier` class.

## Tuned ensemble

In [44]:
airplanes_Xtrain, airplanes_Xvalid, airplanes_ytrain, airplanes_yvalid = train_test_split(airplanes_X_train,
                                                                                          airplanes_y_train,
                                                                                          test_size=0.2,
                                                                                          random_state=20130810)

In [45]:
sc = StandardScaler()

In [46]:
airplanes_scaledXtrain = sc.fit_transform(airplanes_Xtrain)

airplanes_scaledXvalid = sc.transform(airplanes_Xvalid)

In [47]:
w_knn = [1, 2, 3, 4, 5, 6, 7, 8]
w_softmax = [1, 2, 3, 4, 5, 6, 7, 8]
w_gaussiannb = [1, 2, 3, 4, 5, 6, 7, 8]

In [48]:
%%time

best_f1score = 0
best_wts = []

for wt_knn in w_knn:
  for wt_smax in w_softmax:
    for wt_gnb in w_gaussiannb:
      
      learner_vcf = VotingClassifier(estimators=[('knn', learner_best_knn),
                                                 ('softmax', learner_best_softmax),
                                                 ('nb', learner_best_gaussiannb)],
                                     voting='soft',
                                     weights=[wt_knn, wt_smax, wt_gnb])
      
      learner_vcf.fit(airplanes_scaledXtrain, airplanes_ytrain)

      wt_f1score = f1_score(airplanes_yvalid,
                            learner_vcf.predict(airplanes_scaledXvalid),
                            average='macro')
      
      if wt_f1score > best_f1score:
        best_f1score = wt_f1score
        best_wts = [wt_knn, wt_smax, wt_gnb]

CPU times: user 1min 22s, sys: 136 ms, total: 1min 22s
Wall time: 1min 22s


In [49]:
best_f1score

0.7022971938091173

In [50]:
best_wts

[7, 2, 1]

Re-train the best model on the full training data with these final weights

In [51]:
sc = StandardScaler()

airplanes_scaledXtrain = sc.fit_transform(airplanes_X_train)
airplanes_scaledXtest = sc.transform(airplanes_X_test)

In [52]:
learner_best_vcf = VotingClassifier(estimators=[('knn', learner_best_knn),
                                                ('softmax', learner_best_softmax),
                                                ('nb', learner_best_gaussiannb)],
                                    voting='soft',
                                    weights=[7, 2, 1])

In [53]:
learner_best_vcf.fit(airplanes_scaledXtrain,
                     airplanes_y_train)

VotingClassifier(estimators=[('knn',
                              KNeighborsClassifier(algorithm='auto',
                                                   leaf_size=30,
                                                   metric='minkowski',
                                                   metric_params=None,
                                                   n_jobs=None, n_neighbors=8,
                                                   p=2, weights='uniform')),
                             ('softmax',
                              LogisticRegression(C=1, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='multinomial',
                                                 n_jobs=None, penalty='l2',
                                    

In [54]:
f1_score(airplanes_y_test,
         learner_best_vcf.predict(airplanes_scaledXtest),
         average='macro')

0.6868928650231731